In [ ]:
%cd ~/projects/wind/

In [ ]:
import polars as pl
import numpy as np
import pymc as pm
import plotly.express as px
from datetime import datetime, timedelta
import pytensor.tensor as pt
import plotly.graph_objs as go

In [ ]:
bidding_area = "ELSPOT NO3"
dataset_path = "data/windpower_area_dataset.parquet"
val_cutoff = datetime(2025, 1, 1)
data_train = (
    pl.scan_parquet(dataset_path)
    .filter(
        pl.col("bidding_area") == bidding_area,
        pl.col("time_ref") >= datetime(2024, 1, 1),
        pl.col("time_ref") < val_cutoff,
        # pl.col("time") >= pl.col("time_ref").dt.date() + timedelta(days=1),
        # pl.col("time") < pl.col("time_ref").dt.date() + timedelta(days=2),
        (pl.col("time_ref") + timedelta(days=2)).dt.date() == pl.col("time").dt.date(),
    )
    .with_columns(lt=(pl.col("time") - pl.col("time_ref")).dt.total_hours())
)
data_val = (
    pl.scan_parquet(dataset_path)
    .filter(
        pl.col("bidding_area") == bidding_area,
        pl.col("time_ref") >= val_cutoff,
        # pl.col("time") >= pl.col("time_ref").dt.date() + timedelta(days=1),
        # pl.col("time") < pl.col("time_ref").dt.date() + timedelta(days=2),
        (pl.col("time_ref") + timedelta(days=2)).dt.date() == pl.col("time").dt.date(),
    )
    .with_columns(lt=(pl.col("time") - pl.col("time_ref")).dt.total_hours())
)

In [ ]:
max_lt = 62


def get_emos_features(data):
    X_mu = (
        data.select(
            pl.lit(1).alias("intercept"),
            "mean_sum_pred",
            "min_sum_pred",
            "max_sum_pred",
            "pred_lag1",
            "pred_lag2",
            "pred_lead1",
            "pred_lead2",
            "last_power",
            "recent_mean",
            "ramp",
            "recent_max",
            "recent_min",
            "unavailable_transmission",
        )
        .cast(pl.Float32)
        .collect()
        .to_numpy(order="c")
    )

    X_sigma = (
        data.select(
            pl.lit(1).alias("intercept"),
            pl.col("std_sum_pred").log().alias("log_std_sum_pred"),
            (pl.col("lt") / max_lt).alias("lt"),
            "mean_sum_pred",
            (pl.col("max_sum_pred") - pl.col("min_sum_pred"))
            .log()
            .alias("log_range_pred"),
            pl.col("recent_std").log().alias("log_recent_std"),
            pl.col("ramp").abs().log().alias("log_ramp"),
            "unavailable_transmission",
        ).cast(pl.Float32)
        .collect()
        .to_numpy(order="c")
    )

    y = data.select("relative_power").collect().to_numpy()[:, 0]
    sample_weight = data.select("operating_power_max").collect().to_numpy()[:, 0]
    return X_mu, X_sigma, y, sample_weight


X_mu_train, X_sigma_train, y_train, sample_weight_train = get_emos_features(data_train)
X_mu_val, X_sigma_val, y_val, sample_weight_val = get_emos_features(data_val)

In [ ]:
def get_area_capacity(path, times):
    max_capacity = pl.scan_csv(path, try_parse_dates=True)

    area_capacity = (
        times.join(max_capacity, how="cross")
        .filter(pl.col("time") >= pl.col("production_start_date"))
        .group_by(pl.col("time").alias("time_ref"), "bidding_area")
        .agg(
            operating_power_max=pl.col("operating_power_max").sum(),
            mean_production=pl.col("mean_production").sum(),
            num_turbines=pl.col("num_turbines").sum(),
        )
    )
    return area_capacity


windpower = (
    pl.scan_parquet("data/wind_power_per_bidzone.parquet").rename(
        {"__index_level_0__": "time"}
    )
).unpivot(index="time", variable_name="bidding_area", value_name="power")

times = windpower.select(pl.col("time").unique())
area_capacity = get_area_capacity("data/windparks_enriched.csv", times)

In [ ]:
# local_pred = (
#     pl.scan_parquet("data/em0_model_pred.parquet")
#     .filter(pl.col("bidding_area") == bidding_area)
#     .group_by("time_ref", "time", "lt", "bidding_area", "em")
#     .agg(sum_local_pred=pl.col("local_power_pred").sum())
#     .join(area_capacity, on=["time_ref", "bidding_area"])
#     .join(windpower, on=["time", "bidding_area"])
#     .with_columns(
#         sum_local_pred=pl.col("sum_local_pred") / pl.col("operating_power_max"),
#     )
#     .group_by("time_ref", "time", "lt", "bidding_area")
#     .agg(
#         power=pl.col("power").first(),
#         relative_power=pl.col("relative_power").first(),
#         operating_power_max=pl.col("operating_power_max").first(),
#         mean_production=pl.col("mean_production").first(),
#         num_turbines=pl.col("num_turbines").first(),
#         mean_sum_pred=pl.col("sum_local_pred").mean(),
#         std_sum_pred=pl.col("sum_local_pred").std(),
#         min_sum_pred=pl.col("sum_local_pred").min(),
#         max_sum_pred=pl.col("sum_local_pred").max(),
#     )
# )
# local_pred.filter(
#     pl.col("time_ref").dt.date() == datetime(2025, 1, 1), pl.col("lt") == 39
# ).collect()

In [ ]:
from dataclasses import dataclass
from wind.model.pred_area_bayesian import BayesianAreaModel, get_emos_features


@dataclass
class EvalResult:
    name: str
    rmse: float
    crps: float


def per_observation_crps(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    if y_pred.shape[1:] != (1,) * (y_pred.ndim - y_true.ndim - 1) + y_true.shape:
        raise ValueError(
            f"""Expected y_pred to have one extra sample dim on left.
                Actual shapes: {y_pred.shape} versus {y_true.shape}"""
        )

    absolute_error = np.mean(np.abs(y_pred - y_true), axis=0)

    num_samples = y_pred.shape[0]
    if num_samples == 1:
        return absolute_error

    y_pred = np.sort(y_pred, axis=0)
    diff = y_pred[1:] - y_pred[:-1]
    weight = np.arange(1, num_samples) * np.arange(num_samples - 1, 0, -1)
    weight = weight.reshape(weight.shape + (1,) * (diff.ndim - 1))

    return absolute_error - np.sum(diff * weight, axis=0) / num_samples**2


def crps(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    sample_weight: np.ndarray | None = None,
) -> float:
    return float(
        np.average(per_observation_crps(y_true, y_pred), weights=sample_weight)
    )


def evaluate_bayesian(y_true: pl.LazyFrame) -> EvalResult:
    val_cutoff = y_true.select(pl.col("time_ref").min()).collect().item()
    data = pl.scan_parquet("data/windpower_area_dataset.parquet").filter(
        pl.col("time").dt.date() == (pl.col("time_ref") + timedelta(days=2)).dt.date()
    )

    df_train = (
        data.filter(
            pl.col("time_ref") < val_cutoff,
            pl.col("time_ref") >= val_cutoff - timedelta(days=365),
            pl.col("relative_power").is_not_null(),
        )
        .sort("bidding_area", "time_ref", "time")
        .select(
            pl.col("*").fill_null(strategy="forward").over("bidding_area"),
        )
    )

    df_val = (
        y_true.join(data, on=["bidding_area", "time_ref", "time"], how="left")
        .sort("bidding_area", "time_ref", "time")
        .select(
            pl.col("*").fill_null(strategy="forward").over("bidding_area"),
        )
    )

    preds = []
    for bidding_area in ["ELSPOT NO1", "ELSPOT NO2", "ELSPOT NO3", "ELSPOT NO4"]:
        X_mu_train, X_sigma_train, y_train, scaling_factor_train = get_emos_features(
            df_train.filter(pl.col("bidding_area") == bidding_area)
        )
        X_mu_val, X_sigma_val, y_val, scaling_factor_val = get_emos_features(
            df_val.filter(pl.col("bidding_area") == bidding_area)
        )
        print(
            np.isnan(X_mu_train).sum(),
            np.isnan(X_sigma_train).sum(),
            np.isnan(X_mu_val).sum(),
            np.isnan(X_sigma_val).sum(),
        )
        area_model = BayesianAreaModel()
        area_model.fit(X_mu_train, X_sigma_train, y_train, tune=100, draws=100)
        samples = area_model.predict(X_mu_val, X_sigma_val) * np.expand_dims(
            scaling_factor_val, 1
        )
        preds.append(samples)

    pred_samples = np.concat(preds, axis=0)
    print(pred_samples.shape)
    pred_mean = np.mean(pred_samples, axis=1)
    y_true_values = df_val.select("y_true").collect().to_numpy()[:, 0]
    rmse_score = np.sqrt(np.mean((y_true_values - pred_mean) ** 2))
    crps_score = crps(y_true_values, pred_samples)
    return EvalResult("Bayesian Calibrator", rmse_score, crps_score)


def get_eval_set(
    eval_start: datetime = datetime(2025, 1, 1), eval_stop: datetime | None = None
):
    area_power = (
        pl.scan_parquet("data/wind_power_per_bidzone.parquet").rename(
            {"__index_level_0__": "time"}
        )
    ).unpivot(index="time", variable_name="bidding_area", value_name="y_true")

    y_true = (
        area_power.select("bidding_area", pl.col("time").alias("time_ref"))
        .filter(
            pl.col("time_ref") >= eval_start,
            pl.col("time_ref").dt.hour() == 9,
        )
        .join(area_power, on="bidding_area")
        .filter(
            pl.col("time").dt.date()
            == (pl.col("time_ref") + timedelta(days=2)).dt.date()
        )
        .select(
            "bidding_area",
            "time_ref",
            "time",
            (pl.col("time") - pl.col("time_ref")).dt.total_hours().alias("lt"),
            "y_true",
        )
    )
    return y_true


y_true = get_eval_set()

val_cutoff = y_true.select(pl.col("time_ref").min()).collect().item()
data = pl.scan_parquet("data/windpower_area_dataset.parquet").filter(
    pl.col("time").dt.date() == (pl.col("time_ref") + timedelta(days=2)).dt.date()
)

df_train = (
    data.filter(
        pl.col("time_ref") < val_cutoff,
        pl.col("time_ref") >= val_cutoff - timedelta(days=365),
        pl.col("relative_power").is_not_null(),
    )
    .sort("bidding_area", "time_ref", "time")
    .select(
        pl.col("*").fill_null(strategy="forward").over("bidding_area"),
    )
)

df_val = (
    y_true.join(data, on=["bidding_area", "time_ref", "time"], how="left")
    .sort("bidding_area", "time_ref", "time")
    .select(
        pl.col("*").fill_null(strategy="forward").over("bidding_area"),
    )
)

preds = []
# def run_model(bidding_area)

# for bidding_area in ["ELSPOT NO1", "ELSPOT NO2", "ELSPOT NO3", "ELSPOT NO4"]:
for bidding_area in ["ELSPOT NO2"]:
    X_mu_train, X_sigma_train, y_train, scaling_factor_train = get_emos_features(
        df_train.filter(pl.col("bidding_area") == bidding_area)
    )
    X_mu_val, X_sigma_val, y_val, scaling_factor_val = get_emos_features(
        df_val.filter(pl.col("bidding_area") == bidding_area)
    )
    # print(
    #     np.isnan(X_mu_train).sum(),
    #     np.isnan(X_sigma_train).sum(),
    #     np.isnan(X_mu_val).sum(),
    #     np.isnan(X_sigma_val).sum(),
    # )
    # area_model = BayesianAreaModel()
    # area_model.fit(X_mu_train, X_sigma_train, y_train, tune=100, draws=100)
    # samples = area_model.predict(X_mu_val, X_sigma_val) * np.expand_dims(
    #     scaling_factor_val, 1
    # )
    # preds.append(samples)

In [ ]:
def fit_emos_pymc(
    X_mu,
    X_sigma,
    y,
    draws=1500,
    tune=1500,
    target_accept=0.9,
    chains=4,
    seed=123,
):
    with pm.Model() as model:
        Xmu = pm.Data("Xmu", X_mu)
        Xsig = pm.Data("Xsig", X_sigma)
        y_obs = pm.Data("y_obs", y)

        # Priors
        beta = pm.Normal("beta", mu=0.0, sigma=2.0, shape=X_mu.shape[1])
        gamma = pm.Normal("gamma", mu=0.0, sigma=0.2, shape=X_sigma.shape[1])

        # Linear predictors
        mu = pm.Deterministic("mu", pt.dot(Xmu, beta))  # logit-mean
        log_sig = pm.Deterministic("log_sigma", pt.dot(Xsig, gamma))
        sigma = pm.Deterministic("sigma", pt.exp(log_sig))  # > 0

        # Likelihood on capacity factor (0,1) via LogitNormal
        pm.LogitNormal(
            "y", mu=mu, sigma=sigma, observed=y_obs
        )  # bounds handled by dist

        idata = pm.sample(
            draws=draws,
            tune=tune,
            chains=chains,
            cores=chains,
            # target_accept=target_accept,
            random_seed=seed,
            progressbar=True,
            nuts_sampler="nutpie",
            # nuts_sampler_kwargs=dict(backend="jax"),
        )
    return model, idata

In [ ]:
model, idata = fit_emos_pymc(
    X_mu_train,
    X_sigma_train,
    y_train,
    draws=1000,
    tune=1000,
    chains=4,
)

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
import arviz as az
import matplotlib.pyplot as plt

ax = az.plot_trace(idata, var_names="beta", compact=False)
plt.tight_layout()

In [ ]:
with model:
    prior_pred = pm.sample_prior_predictive(draws=10)

In [ ]:
px.histogram(
    pl.DataFrame(
        prior_pred.prior_predictive["y"].stack(sample=("chain", "draw")).values,
        schema=[str(k) for k in range(10)],
    ).unpivot(),
    "value",
)

In [ ]:
px.box(
    pl.concat(
        [
            pl.DataFrame(
                prior_pred.prior_predictive["y"].stack(sample=("chain", "draw")).values,
                schema=[str(k) for k in range(10)],
            ),
            data_train.select("time", "time_ref", "lt").collect(),
        ],
        how="horizontal",
    ).unpivot(index=["time", "time_ref", "lt"]),
    # .filter(pl.col("time") < pl.col("time_ref").dt.date() + timedelta(days=2)),
    "time_ref",
    "value",
)

In [ ]:
def predict_emos_quantiles(
    model, idata, X_mu_new, X_sigma_new, q=(0.025, 0.05, 0.5, 0.95, 0.975)
):
    with model:
        pm.set_data(
            {
                "Xmu": X_mu_new,
                "Xsig": X_sigma_new,
                "y_obs": np.zeros(X_mu_new.shape[0], dtype=np.float32),
            }
        )
        ppc = pm.sample_posterior_predictive(idata)

    # ppc["y"] has shape (n_draws*chains, n_obs)
    posterior = ppc.posterior_predictive["y"].stack(sample=("chain", "draw"))
    samples = posterior.transpose("sample", "y_dim_0").values
    # Quantiles per observation
    qs = np.quantile(samples, q, axis=0).T
    out = pl.DataFrame(qs, schema=[f"q{int(1000 * qq):03d}" for qq in q]).with_columns(
        pred_mean=samples.mean(axis=0),
        pred_std=samples.std(axis=0, ddof=1),
    )
    return (
        out,
        posterior,
    )  # samples  # you can keep samples for scenario generation / ECC later

In [ ]:
out, posterior = predict_emos_quantiles(model, idata, X_mu_val, X_sigma_val)
# out, posterior = predict_emos_quantiles(model, idata, X_mu_train, X_sigma_train)

In [ ]:
df_plot["operating_power_max"]

In [ ]:
df_plot = pl.concat([data_val.collect(), out], how="horizontal").filter(
    pl.col("time") >= datetime(2025, 1, 1),
    pl.col("time") < datetime(2025, 1, 20),
    )
# df_plot = pl.concat([data_train.collect(), out], how="horizontal")

fig = go.Figure(
    [
        go.Scatter(
            name="y_true",
            x=df_plot["time"],
            y=df_plot["relative_power"] * df_plot["operating_power_max"],
            mode="lines",
            line=dict(color="rgb(237, 55, 31)"),
        ),
        go.Scatter(
            name="y_pred",
            x=df_plot["time"],
            y=df_plot["pred_mean"] * df_plot["operating_power_max"],
            mode="lines",
            line=dict(color="rgb(31, 119, 180)"),
        ),
        go.Scatter(
            name="Upper Bound alpha-50",
            x=df_plot["time"],
            y=df_plot["q950"] * df_plot["operating_power_max"],
            mode="lines",
            marker=dict(color="#444"),
            line=dict(width=0),
            showlegend=False,
        ),
        go.Scatter(
            name="Lower Bound alpha-50",
            x=df_plot["time"],
            y=df_plot["q050"] * df_plot["operating_power_max"],
            marker=dict(color="#444"),
            line=dict(width=0),
            mode="lines",
            fillcolor="rgba(68, 68, 68, 0.3)",
            fill="tonexty",
            showlegend=False,
        ),
        # go.Scatter(
        #     name="Upper Bound alpha-5",
        #     x=df_plot["time"],
        #     y=df_plot["q975"],
        #     mode="lines",
        #     marker=dict(color="#444"),
        #     line=dict(width=0),
        #     showlegend=False,
        # ),
        # go.Scatter(
        #     name="Lower Bound alpha-5",
        #     x=df_plot["time"],
        #     y=df_plot["q025"],
        #     marker=dict(color="#444"),
        #     line=dict(width=0),
        #     mode="lines",
        #     fillcolor="rgba(68, 68, 68, 0.15)",

        #     fill="tonexty",
        #     showlegend=False,
        # ),
    ]
)
# [fig.add_vline(x=x) for x in df_plot["time_ref"].unique()]
fig.update_layout(
    yaxis=dict(title=dict(text="Power")),
    # title=dict(text="Continuous, variable value error bars"),
    hovermode="x",
)
fig.update_layout(showlegend=False)
fig.update_layout(
    autosize=False,  # Set to False to manually control width and height
    width=800,       # Set the desired width in pixels
    height=400       # Set the desired height in pixels
)
fig.show()

In [ ]:
df_plot.with_columns(
    # under=pl.col("relative_power") < pl.col("q250"),
    # over=pl.col("relative_power") > pl.col("q750"),
    under=pl.col("relative_power") < pl.col("q025"),
    over=pl.col("relative_power") > pl.col("q975"),
).select(
    pl.col("under").mean(),
    (~pl.col("under") & ~pl.col("over")).mean().alias("in"),
    pl.col("over").mean(),
)

In [ ]:
def per_observation_crps(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    if y_pred.shape[1:] != (1,) * (y_pred.ndim - y_true.ndim - 1) + y_true.shape:
        raise ValueError(
            f"""Expected y_pred to have one extra sample dim on left.
                Actual shapes: {y_pred.shape} versus {y_true.shape}"""
        )

    absolute_error = np.mean(np.abs(y_pred - y_true), axis=0)

    num_samples = y_pred.shape[0]
    if num_samples == 1:
        return absolute_error

    y_pred = np.sort(y_pred, axis=0)
    diff = y_pred[1:] - y_pred[:-1]
    weight = np.arange(1, num_samples) * np.arange(num_samples - 1, 0, -1)
    weight = weight.reshape(weight.shape + (1,) * (diff.ndim - 1))

    return absolute_error - np.sum(diff * weight, axis=0) / num_samples**2


def crps(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    sample_weight: np.ndarray | None = None,
):
    return np.average(per_observation_crps(y_true, y_pred), weights=sample_weight)

In [ ]:
from sklearn.metrics import root_mean_squared_error

capacity = data_val.select("operating_power_max").collect().to_series().to_numpy()
y_true = y_val * capacity
y_pred = posterior.transpose("sample", "y_dim_0").values * capacity

print(f"CRPS: {crps(y_true, y_pred)}")
print(f"RMSE: {root_mean_squared_error(y_true, np.mean(y_pred, axis=0))}")